# Chapter 7.3: Reranking (검색 결과 재정렬)

이 노트북은 OSTEP RAG 시스템에 Reranking 기법을 적용하여 검색 품질을 개선하는 방법을 실습합니다.

## 📚 학습 목표
- Reranking 개념 이해: 초기 검색 결과를 더 정교하게 재평가
- LLM 기반 관련성 점수 부여 방법
- Ollama를 활용한 로컬 LLM 기반 Reranking 구현

## 📋 실습 구성
1) 설정 및 하이퍼파라미터 정의
2) 기존 데이터 로드 및 Ollama 설정
3) Reranking 개념 설명
4) Reranking 함수 구현
5) 비교 실험: 일반 검색 vs Reranking
6) 다양한 쿼리로 Reranking 효과 확인

> **핵심 아이디어**: 초기 검색으로 많은 후보를 빠르게 찾고, LLM으로 각 후보의 관련성을 정교하게 평가하여 최종 결과를 선정하면 검색 정확도가 향상됩니다.


---
## 1️⃣ 설정 및 하이퍼파라미터 정의

이 셀에서는 경로, 모델, 검색 파라미터 등을 정의합니다.

**주요 내용:**
- 기존 산출물 경로 (청크 JSON, FAISS 인덱스)
- Ollama 서버 설정 (Reranking용 LLM)
- Reranking 파라미터 (초기 검색 개수, 최종 선택 개수 등)

**실행 결과:**
- 설정값이 출력됩니다.


In [ ]:
# ========================================
# 1️⃣ 설정 / 하이퍼파라미터
# ========================================
import os
import json
import faiss
import numpy as np
import torch
import requests
from pathlib import Path
from typing import List, Dict, Any, Tuple

# 재현성
SEED = 42
np.random.seed(SEED)

# 디바이스
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 경로 (기존 산출물)
CHUNK_FILE = "OSTEP_RAG/data/chunk/ostep_tok400_ov20.json"
INDEX_FILE = "OSTEP_RAG/data/index/ostep_hnsw.index"

# 검색 하이퍼파라미터
INITIAL_K = 20  # 초기 검색에서 가져올 문서 수 (많이 가져옴)
FINAL_K = 5     # Reranking 후 최종 선택할 문서 수
EF_SEARCH = 64

# 임베딩 모델
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
NORMALIZE = True

# Ollama 설정 (Reranking용 LLM)
OLLAMA_HOST = "http://localhost:11434"
LLM_MODEL = "llama3.1:8b"
TEMPERATURE = 0.1  # 낮은 temperature로 일관된 점수 부여

print(f"🔧 Device: {DEVICE}")
print(f"📁 청크 파일: {CHUNK_FILE}")
print(f"📁 인덱스 파일: {INDEX_FILE}")
print(f"🔎 초기 검색 K={INITIAL_K}, 최종 K={FINAL_K}")
print(f"🧠 임베딩 모델: {EMBED_MODEL}")
print(f"🤖 LLM: {LLM_MODEL} @ {OLLAMA_HOST}")


---
## 2️⃣ 기존 데이터 로드 및 Ollama 설정

Chapter 2-4에서 생성한 청크 JSON과 FAISS 인덱스를 로드하고, Ollama 서버 연결을 확인합니다.

**주요 내용:**
- 청크 데이터 로드
- FAISS 인덱스 로드
- 임베딩 모델 로드
- Ollama 서버 연결 확인

**실행 결과:**
- 데이터 로드 완료 메시지와 Ollama 서버 상태가 출력됩니다.


In [ ]:
# 청크 로드
print("청크 데이터 로드 중...")
with open(CHUNK_FILE, 'r', encoding='utf-8') as f:
    CHUNKS = json.load(f)
print(f"✓ 청크 로드 완료: {len(CHUNKS)}개")

# 인덱스 로드
print("\n인덱스 로드 중...")
INDEX = faiss.read_index(INDEX_FILE)
print(f"✓ 인덱스 로드 완료: {INDEX.ntotal}개 벡터")

# 임베딩 모델 로드
print(f"\n임베딩 모델 로드 중: {EMBED_MODEL}")
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer(EMBED_MODEL, device="cpu" if DEVICE=="cpu" else DEVICE)
print("✓ 임베딩 모델 로드 완료")

# Ollama 서버 연결 확인
print(f"\nOllama 서버 연결 확인: {OLLAMA_HOST}")
try:
    response = requests.get(f"{OLLAMA_HOST}/api/version", timeout=5)
    if response.status_code == 200:
        print(f"✓ Ollama 서버 연결 성공")
    else:
        print(f"⚠️  Ollama 서버 응답 오류: {response.status_code}")
except Exception as e:
    print(f"⚠️  Ollama 서버 연결 실패: {e}")
    print("   Ollama 서버가 실행 중인지 확인하세요: ollama serve")


---
## 3️⃣ Reranking 개념 설명

이 셀에서는 Reranking의 핵심 개념을 설명합니다.

**Reranking이란?**
- **문제**: 초기 검색(임베딩 기반)은 빠르지만 때로 정확도가 낮음
- **해결책**: 2단계 검색 전략
  1. **1단계 (초기 검색)**: 임베딩 유사도로 많은 후보(예: 20개)를 빠르게 검색
  2. **2단계 (Reranking)**: LLM이 각 후보의 관련성을 정교하게 평가하여 재정렬
  3. **최종**: 상위 몇 개(예: 5개)만 선택
- **효과**: 속도와 정확도의 균형을 맞춤

**Reranking의 장점:**
- 초기 검색의 속도 유지
- LLM의 정교한 이해력 활용
- 미묘한 관련성 차이 포착

**실행 결과:**
- 개념 설명이 출력됩니다.


In [ ]:
print("=" * 80)
print("[Reranking 개념]")
print("=" * 80)
print()
print("전통적 검색:")
print("  쿼리 → 임베딩 유사도 검색 → Top-5 결과 반환")
print()
print("Reranking 검색:")
print("  쿼리 → 임베딩 유사도 검색 → Top-20 후보")
print("  ↓")
print("  각 후보에 대해 LLM이 관련성 점수(1-10) 부여")
print("  ↓")
print("  점수 기준으로 재정렬 → Top-5 결과 반환")
print()
print("=" * 80)
print("💡 핵심: 빠른 초기 검색 + 정교한 LLM 평가 = 속도 & 정확도")
print("=" * 80)


---
## 4️⃣ Reranking 함수 구현

이 셀에서는 Ollama를 사용하여 문서 관련성을 평가하고 재정렬하는 함수를 구현합니다.

**주요 내용:**
- LLM 기반 관련성 점수 부여 함수
- 초기 검색 + Reranking 통합 함수
- 샘플 쿼리로 Reranking 테스트

**실행 결과:**
- Reranking 과정과 점수가 출력됩니다.


In [ ]:
def distance_to_similarity(distances: np.ndarray) -> np.ndarray:
    """L2 거리를 유사도로 변환 (정규화된 벡터 가정)"""
    return 1.0 - (distances / 2.0)

def initial_search(query: str, k: int = INITIAL_K):
    """초기 검색 (임베딩 기반)"""
    faiss.ParameterSpace().set_index_parameter(INDEX, "efSearch", EF_SEARCH)
    q_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=NORMALIZE).astype('float32')
    D, I = INDEX.search(q_emb, k)
    scores = distance_to_similarity(D[0])
    return I[0], scores

def ollama_score_relevance(query: str, document: str) -> float:
    """
    LLM을 사용하여 문서의 쿼리 관련성을 1-10 점수로 평가합니다.
    
    Args:
        query: 사용자 질문
        document: 평가할 문서
    
    Returns:
        관련성 점수 (1-10)
    """
    prompt = f"""Rate the relevance of the following document to the query on a scale of 1-10.
Consider how well the document answers or relates to the query.
Output ONLY a single number between 1 and 10, nothing else.

Query: {query}

Document: {document[:500]}

Relevance score (1-10):"""
    
    payload = {
        "model": LLM_MODEL,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": TEMPERATURE,
            "num_predict": 10,
        }
    }
    try:
        r = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload, timeout=30)
        r.raise_for_status()
        data = r.json()
        response_text = data.get("response", "").strip()
        # 숫자만 추출
        score_str = ''.join(filter(lambda x: x.isdigit() or x == '.', response_text.split()[0] if response_text.split() else "5"))
        score = float(score_str) if score_str else 5.0
        # 1-10 범위로 클램핑
        score = max(1.0, min(10.0, score))
        return score
    except Exception as e:
        print(f"  [Warning] LLM 점수 부여 실패: {e}, 기본값 5.0 사용")
        return 5.0

def rerank_documents(query: str, indices: np.ndarray, initial_scores: np.ndarray, top_k: int = FINAL_K) -> Tuple[List[int], List[float]]:
    """
    LLM을 사용하여 문서를 재정렬합니다.
    
    Args:
        query: 사용자 질문
        indices: 초기 검색 문서 인덱스
        initial_scores: 초기 검색 점수
        top_k: 최종 선택할 문서 수
    
    Returns:
        재정렬된 인덱스와 LLM 점수
    """
    print(f"Reranking {len(indices)}개 문서...")
    
    reranked = []
    for i, idx in enumerate(indices):
        if idx < 0:
            continue
        chunk = CHUNKS[idx]
        llm_score = ollama_score_relevance(query, chunk['text'])
        reranked.append((idx, llm_score, initial_scores[i]))
        if (i + 1) % 5 == 0:
            print(f"  진행: {i+1}/{len(indices)}")
    
    # LLM 점수로 정렬
    reranked.sort(key=lambda x: x[1], reverse=True)
    
    # Top-K 선택
    top_indices = [x[0] for x in reranked[:top_k]]
    top_scores = [x[1] for x in reranked[:top_k]]
    
    return top_indices, top_scores

# 샘플 테스트
print("=" * 80)
print("[Reranking 테스트]")
print("=" * 80)

test_query = "How does the operating system manage virtual memory?"
print(f"\n쿼리: {test_query}")

print("\n1단계: 초기 검색 (Top-20)")
indices_initial, scores_initial = initial_search(test_query, k=INITIAL_K)
print(f"✓ 초기 검색 완료: {len(indices_initial)}개 후보")

print(f"\n2단계: LLM Reranking (Top-{FINAL_K} 선택)")
indices_reranked, scores_reranked = rerank_documents(test_query, indices_initial, scores_initial, top_k=FINAL_K)
print(f"✓ Reranking 완료")

print(f"\n재정렬 결과:")
for rank, (idx, score) in enumerate(zip(indices_reranked, scores_reranked), 1):
    chunk = CHUNKS[idx]
    print(f"\n{rank}. LLM Score: {score:.1f}/10")
    print(f"   Chapter: {chunk.get('chapter_title', 'N/A')}")
    print(f"   Preview: {chunk['text'][:100]}...")


---
## 5️⃣ 비교 실험: 일반 검색 vs Reranking

이 셀에서는 동일한 쿼리에 대해 일반 검색과 Reranking을 비교합니다.

**주요 내용:**
- 일반 검색: Top-5 직접 선택
- Reranking: Top-20 후보에서 LLM으로 Top-5 선택
- 결과 비교

**실행 결과:**
- 두 방식의 검색 결과와 순위 차이가 출력됩니다.


In [ ]:
TEST_QUERY = "What is the role of the scheduler in an operating system?"

print("=" * 80)
print(f"테스트 쿼리: {TEST_QUERY}")
print("=" * 80)

# 1) 일반 검색 (Top-5)
print("\n[1. 일반 검색 (Top-5 직접 선택)]")
print("-" * 80)
indices_normal, scores_normal = initial_search(TEST_QUERY, k=5)
for rank, (idx, score) in enumerate(zip(indices_normal, scores_normal), 1):
    if idx >= 0:
        chunk = CHUNKS[idx]
        print(f"\n{rank}. 임베딩 유사도: {score:.4f}")
        print(f"   Chapter: {chunk.get('chapter_title', 'N/A')}")
        print(f"   Preview: {chunk['text'][:100]}...")

# 2) Reranking (Top-20 → LLM 평가 → Top-5)
print("\n" + "=" * 80)
print("[2. Reranking (Top-20 후보에서 LLM으로 Top-5 선택)]")
print("-" * 80)
indices_initial_20, scores_initial_20 = initial_search(TEST_QUERY, k=INITIAL_K)
indices_rerank, scores_rerank = rerank_documents(TEST_QUERY, indices_initial_20, scores_initial_20, top_k=5)

for rank, (idx, score) in enumerate(zip(indices_rerank, scores_rerank), 1):
    chunk = CHUNKS[idx]
    # 초기 순위 찾기
    initial_rank = np.where(indices_initial_20 == idx)[0][0] + 1 if idx in indices_initial_20 else -1
    print(f"\n{rank}. LLM 점수: {score:.1f}/10 (초기 순위: {initial_rank})")
    print(f"   Chapter: {chunk.get('chapter_title', 'N/A')}")
    print(f"   Preview: {chunk['text'][:100]}...")

print("\n" + "=" * 80)
print("[비교 결과]")
print("=" * 80)
print(f"일반 검색 Top-1: {CHUNKS[indices_normal[0]].get('chapter_title', 'N/A')[:50]}")
print(f"Reranking Top-1: {CHUNKS[indices_rerank[0]].get('chapter_title', 'N/A')[:50]}")
print("\n💡 Reranking은 초기 순위가 낮았던 문서도 관련성이 높으면")
print("   상위로 올릴 수 있습니다.")


---
## 6️⃣ 다양한 쿼리로 Reranking 효과 확인

이 셀에서는 여러 쿼리로 Reranking의 효과를 확인합니다.

**주요 내용:**
- 다양한 유형의 쿼리 테스트
- 순위 변동 분석

**실행 결과:**
- 각 쿼리별 순위 변동이 출력됩니다.


In [ ]:
# 추가 테스트 쿼리 (간소화 - 1개만 테스트)
test_query = "Explain the concept of paging in memory management"

print("=" * 80)
print(f"쿼리: {test_query}")
print("=" * 80)

# 초기 검색
indices_init, scores_init = initial_search(test_query, k=INITIAL_K)
print(f"\n초기 검색 Top-3:")
for i in range(3):
    if indices_init[i] >= 0:
        print(f"{i+1}. {CHUNKS[indices_init[i]].get('chapter_title', 'N/A')[:60]}")

# Reranking
indices_rerank, scores_rerank = rerank_documents(test_query, indices_init, scores_init, top_k=3)
print(f"\nReranking 후 Top-3:")
for i, (idx, score) in enumerate(zip(indices_rerank, scores_rerank)):
    initial_rank = np.where(indices_init == idx)[0][0] + 1
    print(f"{i+1}. {CHUNKS[idx].get('chapter_title', 'N/A')[:60]} (초기: {initial_rank}위)")

print("\n✅ Reranking 테스트 완료!")


---
## 7️⃣ 요약 및 다음 단계

### 학습 내용 요약

1. **Reranking 개념**: 초기 검색으로 많은 후보를 찾고, LLM으로 정교하게 재평가
2. **효과**: 임베딩 기반 초기 검색의 한계를 LLM의 이해력으로 보완
3. **구현**: Ollama를 활용한 로컬 LLM 기반 관련성 점수 부여
4. **검증**: 초기 순위가 낮았던 관련 문서를 상위로 끌어올림

### Reranking의 장점

- ✅ 초기 검색의 속도 유지
- ✅ LLM의 정교한 문맥 이해 활용
- ✅ 미묘한 관련성 차이 포착
- ✅ 잘못된 초기 순위 보정

### Reranking의 고려사항

- ⚠️  LLM 호출로 인한 추가 레이턴시
- ⚠️  초기 검색 범위(k)와 비용 간 트레이드오프
- ⚠️  LLM 점수 부여의 일관성 확보 필요

### 다음 단계

- **Chapter 7.4**: Self-RAG (자기 평가 기반 검색)
- **Chapter 7.5**: 모든 기법 통합 (CCH + HyDE + Reranking + Self-RAG)

### 주요 함수

- `initial_search(query, k)`: 임베딩 기반 초기 검색
- `ollama_score_relevance(query, document)`: LLM 기반 관련성 점수 부여
- `rerank_documents(query, indices, scores, top_k)`: 문서 재정렬
